In [1]:
from MultiLabelClassificationModel import MultiLabelClassificationModel, TOPICS, RANDOM_SEED

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MultiLabelClassificationModel()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_rela

MultiLabelClassificationModel ready to use


In [3]:
import json 
import pandas as pd
import numpy as np
import re

LOCAL_PATH = "/Users/abraham/Competitive-programming-problems-classification/backend/"
DRIVE_PATH = "/content/drive/MyDrive/Competitive-programming-problems-classification/Github/Competitive-programming-problems-classification/backend"

PATH = LOCAL_PATH

problems = []
for topic in TOPICS:
  codeforcesProblems = json.load(open(f"{PATH}/data/codeforces-{topic}.json"))
  problems.extend(codeforcesProblems)
  # omegaupProblems = json.load(open(f"data/omegaup-{topic}.json"))
  # problems.extend(omegaupProblems)

data = []
for problem in problems:
    validTopics = [topic for topic in problem['topics'] if topic in TOPICS]
    text = problem['history']
    
    data.append({
      'text': text,
      'category': validTopics,
      'url': problem['url'],
    })

df = pd.DataFrame(data)
df.head()

,text,category,url
0,You are participating in Yet Another Tournamen...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1783/C
1,An array a is called ugly if it contains at le...,"[math, sortings]",https://codeforces.com/problemset/problem/1783/A
2,Let' s call a string balanced if all character...,"[greedy, sortings, strings]",https://codeforces.com/problemset/problem/1781/C
3,A company of n people is planning a visit to t...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1781/B
4,This is an interactive problem. Anya has gathe...,"[graphs, greedy, sortings]",https://codeforces.com/problemset/problem/1779/E


In [4]:
from MultiLabelClassificationModel import mlb

binary_categories = mlb.fit_transform(df['category'])
binary_categories

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [5]:
df['binary-category'] = [x.reshape(-1) for x in binary_categories]
df['binary-category'] = df['binary-category'].apply(lambda x: x.reshape(-1))

df.head()

,text,category,url,binary-category
0,You are participating in Yet Another Tournamen...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1783/C,"[0, 0, 0, 1, 0, 0, 1, 0]"
1,An array a is called ugly if it contains at le...,"[math, sortings]",https://codeforces.com/problemset/problem/1783/A,"[0, 0, 0, 0, 1, 0, 1, 0]"
2,Let' s call a string balanced if all character...,"[greedy, sortings, strings]",https://codeforces.com/problemset/problem/1781/C,"[0, 0, 0, 1, 0, 0, 1, 1]"
3,A company of n people is planning a visit to t...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1781/B,"[0, 0, 0, 1, 0, 0, 1, 0]"
4,This is an interactive problem. Anya has gathe...,"[graphs, greedy, sortings]",https://codeforces.com/problemset/problem/1779/E,"[0, 0, 1, 1, 0, 0, 1, 0]"


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['binary-category'], test_size=0.1, random_state=RANDOM_SEED, shuffle=True)

In [7]:
x_test.head(), y_test.head()

(5277    Let us call two integers x and y adjacent if l...
 748     A k- multiple free set is a set of integers wh...
 1608    You are given some text t and a set of n strin...
 4012    You are given two integers l and r , l≤ r . Fi...
 5946    In Ancient Berland there were n cities and m t...
 Name: text, dtype: object,
 5277    [0, 0, 1, 0, 1, 1, 0, 0]
 748     [0, 0, 0, 1, 0, 0, 1, 0]
 1608    [1, 0, 0, 1, 0, 0, 0, 1]
 4012    [0, 0, 0, 1, 1, 0, 0, 0]
 5946    [0, 0, 1, 0, 0, 0, 0, 0]
 Name: binary-category, dtype: object)

In [8]:

x_test.shape, y_test.shape

((775,), (775,))

In [13]:
from multiprocessing import Pool

PROCESSES = 15

with Pool(processes=PROCESSES) as pool:
  y_pred = pool.map(model.predict, x_test)

In [1]:
from data.testing_results import y_test_results

In [2]:
y_pred = y_test_results

In [3]:
y_pred[0]

[{'topic': 'math', 'probability': 92.52},
 {'topic': 'number theory', 'probability': 92.2}]

In [13]:
y_pred_binary = []
for pred in y_pred:
  y_pred_binary.append([values['topic'] for values in pred])
    
y_pred_binary = mlb.fit_transform(y_pred_binary)
y_pred_binary

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0]])

In [14]:
y_pred_binary = np.asarray(y_pred_binary)
y_pred_binary.shape

(775, 8)

In [15]:
y_pred_binary

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0]])

In [16]:
y_test_fixed = [y.tolist() for y in y_test]

In [17]:
from sklearn.metrics import classification_report

report = classification_report(
  y_test_fixed,
  y_pred_binary,
  output_dict = False,
  target_names = TOPICS
)

/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
print(report)

                 precision    recall  f1-score   support

       sortings       0.64      0.64      0.64       250
        strings       0.81      0.71      0.76        55
         greedy       0.62      0.67      0.64       143
  number theory       0.62      0.76      0.68       330
           math       0.74      0.74      0.74       291
         graphs       0.77      0.66      0.71       114
       geometry       0.50      0.28      0.36       187
data structures       0.79      0.77      0.78        95

      micro avg       0.67      0.65      0.66      1465
      macro avg       0.69      0.65      0.66      1465
   weighted avg       0.66      0.65      0.65      1465
    samples avg       0.67      0.67      0.63      1465



In [21]:
from sklearn.metrics import multilabel_confusion_matrix

multiMat = multilabel_confusion_matrix(
  y_test_fixed,
  y_pred_binary,
)

for topic, mat in zip(TOPICS, multiMat):
  print(f"Multilabel confusion matrix {topic}")
  print(mat)

Multilabel confusion matrix sortings
[[434  91]
 [ 91 159]]
Multilabel confusion matrix strings
[[711   9]
 [ 16  39]]
Multilabel confusion matrix greedy
[[572  60]
 [ 47  96]]
Multilabel confusion matrix number theory
[[290 155]
 [ 80 250]]
Multilabel confusion matrix math
[[410  74]
 [ 77 214]]
Multilabel confusion matrix graphs
[[638  23]
 [ 39  75]]
Multilabel confusion matrix geometry
[[537  51]
 [135  52]]
Multilabel confusion matrix data structures
[[661  19]
 [ 22  73]]
